In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import time
import numpy as np
import tensorflow as tf
from VGG16_GAP import VGG16_GAP

In [ ]:
import numpy as np
import pandas as pd
import skimage.io as imageio
import pickle

In [ ]:
from progress.bar import Bar
from ipywidgets import IntProgress
from IPython.display import display

In [ ]:
with open('save/label_dict.pkl', 'rb') as f:
    y_dict = pickle.load(f)

In [ ]:
with open('save/inv_label_dict.pkl', 'rb') as f:
    inv_y_dict = pickle.load(f)

In [ ]:
TEST_DIR = "/home/cmchang/DLCV2018SPRING/final/dlcv_final_2_dataset/test/"

In [ ]:
test_list = list()
for root, subdir, files in os.walk(TEST_DIR):
    for file in sorted(files):
        if '.jpg' in file:
            test_list.append(os.path.join(TEST_DIR, file))

In [ ]:
def readImgList(file_list):
    images = list()
    for i, file in enumerate(file_list):
        print(i, end="\r")
        img = imageio.imread(file)
        img = img.astype(int)
        images.append(img)
    return np.array(images)

In [ ]:
def transformLabel(id_list, y_dict):
    label = list()
    for uid in list(id_list):
        label.append(y_dict[uid])
    return np.array(label)

In [ ]:
def one_hot_encoding(class_numbers, num_classes):
    return np.eye(num_classes, dtype=float)[class_numbers]

In [ ]:
def initialize_uninitialized(sess):
    global_vars = tf.global_variables()
    is_not_initialized = sess.run([tf.is_variable_initialized(var) for var in global_vars])
    not_initialized_vars = [v for (v,f) in zip(global_vars, is_not_initialized) if not f]
    if len(not_initialized_vars): 
            sess.run(tf.variables_initializer(not_initialized_vars))

In [ ]:
Xtest = readImgList(test_list)

In [ ]:
scope_name = "Model"

In [ ]:
model = VGG16_GAP(scope_name=scope_name)

In [ ]:
model.build(vgg16_npy_path= para_dict16,#init_model,
            shape=Xtest.shape[1:],
            classes=len(y_dict),
            conv_pre_training=True,
            fc_pre_training=True,
            new_bn=False)

In [ ]:
dp = [1.0]
model.set_idp_operation(dp=dp)

In [ ]:
import numpy as np

def count_number_params(para_dict):
    n = 0
    for k,v in sorted(para_dict.items()):
        if 'bn_mean' in k:
            continue
        elif 'bn_variance' in k:
            continue
        elif 'gamma' in k:
            continue
        elif 'beta' in k:
            continue
        elif 'conv' in k or 'fc' in k:
            n += get_params_shape(v[0].shape.as_list())
            n += get_params_shape(v[1].shape.as_list())
    return n

def get_params_shape(shape):
    n = 1
    for dim in shape:
        n = n*dim
    return n

def count_flops(para_dict, net_shape, input_shape=(3, 218, 178)):
    # Format:(channels, rows,cols)
    total_flops_per_layer = 0
    input_count = 0
    for k,v in sorted(para_dict.items()):
        if 'bn_mean' in k:
            continue
        elif 'bn_variance' in k:
            continue
        elif 'gamma' in k:
            continue
        elif 'beta' in k:
            continue
        elif 'fc' in k:
            continue
        elif 'conv' in k:
            conv_filter = v[0].shape.as_list()[3::-1] # (64 ,3 ,3 ,3)  # Format: (num_filters, channels, rows, cols)
            stride = 1
            padding = 1

            if conv_filter[1] == 0:
                n = conv_filter[2] * conv_filter[3] # vector_length
            else:
                n = conv_filter[1] * conv_filter[2] * conv_filter[3]  # vector_length

            flops_per_instance = n + ( n -1)    # general defination for number of flops (n: multiplications and n-1: additions)

            num_instances_per_filter = (( input_shape[1] - conv_filter[2] + 2 * padding) / stride) + 1  # for rows
            num_instances_per_filter *= ((input_shape[2] - conv_filter[3] + 2 * padding) / stride) + 1  # multiplying with cols

            flops_per_filter = num_instances_per_filter * flops_per_instance
            total_flops_per_layer += flops_per_filter * conv_filter[0]  # multiply with number of filters

            total_flops_per_layer += conv_filter[0] * input_shape[1] * input_shape[2] # bias

            input_shape = net_shape[input_count].as_list()[3:0:-1]
            input_count +=1

    total_flops_per_layer += net_shape[-1].as_list()[1] *2360*2
    return total_flops_per_layer

def countFlopsParas(net):
    total_flops = count_flops(net.para_dict, net.net_shape)
    if total_flops / 1e9 > 1:   # for Giga Flops
        print(total_flops/ 1e9 ,'{}'.format('GFlops'))
    else:
        print(total_flops / 1e6 ,'{}'.format('MFlops'))

    total_params = count_number_params(net.para_dict)

    if total_params / 1e9 > 1:   # for Giga Flops
        print(total_params/ 1e9 ,'{}'.format('G'))
    else:
        print(total_params / 1e6 ,'{}'.format('M'))
    
    return total_flops, total_params

In [ ]:
flops, params = countFlopsParas(model)
print("Flops: %3f M, Paras: %3f M" % (flops/1e6, params/1e6))
FLAG_flops_M = flops/1e6
FLAG_params_M = params/1e6

In [ ]:
flops, params = countFlopsParas(model)
print("Flops: %3f M, Paras: %3f M" % (flops/1e6, params/1e6))
FLAG_flops_M = flops/1e6
FLAG_params_M = params/1e6

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.global_variables())
    print("Initialized")
    output = []
    for dp_i in dp:
        for i in range(int(Xtest.shape[0]/200+1)):
            print(i, end="\r")
            st = i*200
            ed = min((i+1)*200, Xtest.shape[0])
            prob = sess.run(model.prob_dict[str(int(dp_i*100))], feed_dict={model.x: Xtest[st:ed,:], 
                                                                        model.is_train: False})
            output.append(prob)

In [ ]:
pred_prob = np.concatenate(output)
pred_class = np.argmax(pred_prob, 1)
final_id = list()
for pred in pred_class:
    final_id.append(inv_y_dict[pred])

In [ ]:
doutput = pd.DataFrame({'id':np.arange(len(final_id))+1,
              'ans':final_id}, columns=['id','ans'])

In [ ]:
doutput.to_csv("pred_val8647_final_newCL_v5_lambda-1e-1_dynamic_gap_L5_v3_rescale0-1_save_linear.csv",index=False)

## Gamma 

In [ ]:
a = np.load("/home/cmchang/DLCV2018SPRING/final/finetune_gap_v3_rescale0-1_save_linear/sparse_dict.npy", encoding='latin1').item()

In [ ]:
gamma = list()
for k, v in sorted(a.items()):
    if 'gamma' in k:
        mystr = k+',"'
        for i in a[k]:
            mystr = mystr+str(i)+","
        mystr = mystr+'"'
        # print(mystr)
        gamma.append(mystr)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.plot(range(len(a['conv1_1_gamma'])), a['conv1_1_gamma'], label="conv1")
plt.plot(range(len(a['conv2_1_gamma'])), a['conv2_1_gamma'], label="conv2")
plt.plot(range(len(a['conv3_1_gamma'])), a['conv3_1_gamma'], label="conv3")
plt.plot(range(len(a['conv4_1_gamma'])), a['conv4_1_gamma'], label="conv4")
plt.plot(range(len(a['conv5_1_gamma'])), a['conv5_1_gamma'], label="conv5")
plt.legend()

In [ ]:
with open(os.path.join(FLAG_save_dir, "gamma.csv"),'a') as resultFile:
    wr = csv.writer(resultFile)
    wr.writerow(gamma)

##  Check performance on validation set

In [ ]:
HOME_DIR = "/home/cmchang/DLCV2018SPRING/final/"
VALID_DIR = HOME_DIR+"dlcv_final_2_dataset/val/"

dvalid = pd.read_csv(HOME_DIR+"dlcv_final_2_dataset/val_id.txt", header=None,sep=" ", names=["img", "id"])

valid_list = list(VALID_DIR+dvalid.img)
Xvalid = readImgList(valid_list)

yvalid = transformLabel(list(dvalid.id), y_dict)
Yvalid = one_hot_encoding(yvalid, len(y_dict))

In [ ]:
batch_size = 200
with tf.Session() as sess:
    with tf.variable_scope(name_or_scope=scope_name) as scop:
        sess.run(tf.global_variables_initializer())
         # validation
        val_accu = 0
        for i in range(int(Xvalid.shape[0]/batch_size)):
            print(i, end='\r')
            st = i*batch_size
            ed = (i+1)*batch_size
            accu = sess.run(model.accu_dict['100'],
                                feed_dict={model.x: Xvalid[st:ed,:],
                                            model.y: Yvalid[st:ed,:],
                                            model.is_train: False})
            val_accu += accu
        val_accu = val_accu/int(Xvalid.shape[0]/batch_size)
        print("val accu : %.4f" % ( val_accu))
    

## GammaSparsify testing

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
FLAG_save_dir = "/home/cmchang/DLCV2018SPRING/final/finetune_gap_v3_rescale0-1_save_linear/"
a = np.load(FLAG_save_dir+"para_dict.npy", encoding="latin1").item()
th = 5e-2

In [ ]:
def gammaSparsifyVGG16(para_dict, thresh=0.5):
    last = None
    sparse_dict = {}
    N_total, N_remain = 0., 0.
    for k, v in sorted(para_dict.items()):
        if 'gamma' in k:
            # trim networks based on gamma
            gamma = v                      
            this = np.where(np.abs(gamma) > thresh)[0]
            sparse_dict[k] = gamma[this] 
            
            # get the layer name
            key = str.split(k,'_gamma')[0]
            
            # trim conv
            conv_, bias_ = para_dict[key]
            conv_ = conv_[:,:,:,this]
            if last is not None:
                conv_ = conv_[:,:,last,:]
            bias_ = bias_[this]
            sparse_dict[key] = [conv_, bias_]
            
            # get corresponding beta, bn_mean, bn_variance
            sparse_dict[key+"_beta"] = para_dict[key+"_beta"][this]
            sparse_dict[key+"_bn_mean"] = para_dict[key+"_bn_mean"][this]
            sparse_dict[key+"_bn_variance"] = para_dict[key+"_bn_variance"][this]
            
            # update
            last = this
            print('%s from %s to %s : %s ' % (k, len(gamma), len(this), len(this)/len(gamma)))
            N_total += len(gamma)
            N_remain += len(this)
    print('sparsify %s percentage' % (N_remain/N_total))
    W_, b_ = para_dict['fc_2']
    W_ = W_[last,:]
    sparse_dict['fc_2'] = [W_, b_]
    return sparse_dict, N_remain/N_total

In [ ]:
b, r = gammaSparsifyVGG16(para_dict=a, thresh=th)

In [ ]:
np.save(os.path.join(FLAG_save_dir, "sparse_dict_"+str(th)+".npy"), b)